In [62]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from dipy.sims.voxel import multi_tensor, multi_tensor_odf
from dipy.data import get_sphere
from dipy.core.sphere import disperse_charges, Sphere, HemiSphere
from dipy.reconst.shm import sh_to_sf_matrix
from dipy.core.gradients import gradient_table
from dipy.reconst.shm import CsaOdfModel, QballModel

from fury import actor, window
from IPython.core.display import Image
from PIL import Image as PILImage

import os
import sys
sys.path.insert(0,'..')
from tqdm import tqdm

In [4]:
J = np.load('/home/brysongray/outputs/fiber-analysis_outputs/hist_to_mri_jacobian.npy')
disp = np.load('/home/brysongray/outputs/fiber-analysis_outputs/hist_to_mri_disp.npy')
odfs = np.load('/home/brysongray/outputs/fiber-analysis_outputs/human_amyg_csd_odfs_patch-55.npy')
sh = np.load('/home/brysongray/outputs/fiber-analysis_outputs/human_amyg_csd_shm_coeff_patch-55.npy')

In [5]:
# helper functions for visualization
WINDOW_SIZE = (400, 400)
SAVEIM_FOLDER = 'images'
if not os.path.exists(SAVEIM_FOLDER):
    os.mkdir(SAVEIM_FOLDER)

def screenshot_animated_sf(sf, sphere, B=None, rot=True, norm=True, scale=True, title='Modeling', theta_step=30):
    '''
    Render a spherical function to file. Returns path to image.
    '''
    
    scene = window.Scene()
    scene.background(window.colors.white)
    
    sf_actor = actor.odf_slicer(sf[None, None, None, :],
                               sphere=sphere, colormap='jet',
                               norm=norm)#, radial_scale=scale)
    
#     sf_actor = actor.odf_slicer(sf,
#                            sphere=sphere, colormap='jet',
#                            norm=norm, B_matrix=B)#, radial_scale=scale)
        
    if rot:
        sf_actor.RotateX(-90)
#         sf_actor.RotateY(180)
        sf_actor.RotateZ(180)
    scene.add(sf_actor)
    
    images = []
    n_frames = 360//theta_step
    for i in np.arange(n_frames):
#         sf_actor.RotateX(theta_step)
#         sf_actor.RotateY(theta_step)
        sf_actor.RotateZ(theta_step)
        scene.reset_clipping_range()
        images.append(PILImage.fromarray(window.snapshot(scene, size=WINDOW_SIZE)))
        
    frame_duration = 15000 // theta_step
    filename = os.path.join(SAVEIM_FOLDER, '{0}.gif'.format(title))
    images[0].save(filename, save_all=True, append_images=images[1:],
                  duration=frame_duration, optimize=False, loop=0)
    scene.clear()

    return filename
    
def screenshot_gradients(sph_gtab, title='Modeling'):
    scene = window.Scene()
    scene.background(window.colors.white)

    scene.add(actor.point(sph_gtab.vertices, window.colors.green, point_radius=0.05))

    outfile = os.path.join(SAVEIM_FOLDER, '{0}.png'.format(title))
    window.snapshot(scene, size=WINDOW_SIZE, fname=outfile)

    scene.clear()
    return outfile

def display_slice(odfs, sphere, norm=True, B=None):
    '''
    odfs : odf field as spherical harmonics
    B : spherical harmonic to spherical function matrix
    sphere : dipy Sphere
    norm : normalizes so the maximum ODF amplitude per voxel is 1
    '''
    scene = window.Scene()
    scene.background(window.colors.white)
    
    sf_actor = actor.odf_slicer(odfs,
                               sphere=sphere, colormap='jet',
                               norm=norm, B_matrix=B)#, radial_scale=scale)
    scene.add(sf_actor)
    
    window.show(scene, size=WINDOW_SIZE)
    
    return scene

In [34]:
sphere = get_sphere('symmetric724')
print(sphere.vertices[0])
print(sphere.x[0])
print(np.sin(sphere.theta[0])*np.cos(sphere.phi[0]))
print(sphere.y[0])
print(np.sin(sphere.theta[0])*np.sin(sphere.phi[0]))
print(sphere.z[0])
print(np.cos(sphere.theta[0]))


[ 5.25406812e-02 -9.98618785e-01  6.12323400e-17]
0.05254068121991737
0.05254068121991737
-0.9986187845303869
-0.9986187845303869
6.123233995736766e-17
6.123233995736766e-17


In [48]:
odfs.shape

(110, 120, 724)

In [49]:
J.shape

(293, 216, 270, 3, 3)

In [50]:
Jvertices = sphere.vertices[None,None,None] @ J

MemoryError: Unable to allocate 277. GiB for an array with shape (293, 216, 270, 724, 3) and data type float64

In [61]:
sh.shape

(110, 120, 15)

In [22]:
print(sphere.theta.shape)
print(sphere.phi.shape)

array([[-1.48985291,  0.52262584,  0.61682709],
       [-0.03367662, -0.77712089,  0.52120405],
       [ 0.29160976,  0.51220312,  0.73977835]])

In [53]:
scale_factor = get_rescale(J[0,100,100],sphere)

In [55]:
odf_ = odfs[0,100,100]*scale_factor

In [60]:
odfs_flat = odfs.reshape(-1,odfs.shape[-1])
odfs_flat.shape

(13200, 724)

In [52]:
def cart_to_polar(X):
    
    x,y,z = X[..., :]
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    phi = np.arctan(y/x)
    
    P = np.stack((r,theta,phi),-1)
    
    return P

def transform_odf(J, odf, sphere):

    theta = sphere.theta
    phi = sphere.phi
    v_ = (sphere.vertices @ J).T
    r_ = np.linalg.norm(v_)
    x_, y_, z_ = v_
    theta_ = np.arccos(z_/r_)
    J_1 = np.stack((np.sin(theta)*np.cos(phi), np.cos(theta)*np.cos(phi), -np.sin(theta)*np.sin(phi),
                    np.sin(theta)*np.sin(phi), np.cos(theta)*np.sin(phi), np.sin(theta)*np.cos(phi),
                    np.cos(theta), -np.sin(theta), np.zeros(len(theta))), axis=-1).reshape((len(theta), 3, 3))
    J_3 = np.stack((x_/r_, y_/r_, z_/r_,
                    x_*z_/np.sqrt(x_**2 + y_**2*r_**2), y_*z_/np.sqrt(x_**2 + y_**2*r_**2), -np.sqrt(x_**2 + y_**2)/r_**2,
                    -y_/(x_**2 + y_**2), x_/(x_**2 + y_**2), np.zeros(len(theta))), axis=-1).reshape((len(theta), 3, 3))
    
    J_polar = J_3 @ J @ J_1
    scale_factor = np.sin(theta)/np.sin(theta_) * 1/np.abs(np.linalg.det(J_polar[...,1:,1:]))
    odf_ = odf*scale_factor

    return odf_, v_
    

def transform_sh_img(sh, J):
    ''' Transform a spherical harmonic image by a Jacobian valued image
    For algorithm details see:
    
    Hong, X., Arlinghaus, L. R., & Anderson, A. W. (2009).
    Spatial normalization of the fiber orientation distribution based on high angular resolution diffusion imaging data.
    Magnetic resonance in medicine, 61(6), 1520–1527. https://doi.org/10.1002/mrm.21916

    Parameters
    ----------
    sh : spherical harmonics
    J : Jacobian

    Returns
    -------
    sh_transformed : spherical harmonic coefficients transformed by J
    '''

    degree = {1:0,
            2:6,
            15:4,
            28:6,
            45:8,
            66:10,
            91:12,
            120:14}
    n = degree[sh.shape[-1]]
    print(f'spherical harmonic degree is {n}.')
    arr_shape = sh.shape[:-1]
    sh = sh.reshape(-1,sh.shape[-1])
    J = J.reshape(-1,3,3)
    sphere = get_sphere('symmetric362')
    B, invB = sh_to_sf_matrix(sphere, n)
    # transform each odf sequentially because of memory constraints
    sh_transformed = np.zeros_like(sh)
    for i in range(len(sh)):
        # transform from spherical harmonics to spherical function
        odf = np.dot(sh[i], B)
        # transform vectors
        odf, vertices = transform_odf(J[i], odf, sphere)
        # transform back into spherical harmonics
        B, invB = sh_to_sf_matrix(vertices, n)
        sh_transformed[i] = np.dot(invB.T, odf)
    sh_transformed = sh_transformed.reshape(arr_shape+sh.shape[-1])
    
    return sh_transformed

In [8]:
print(f"odfs shape & dtype: {odfs.shape}, {odfs.dtype}")
print(f"sphere.vertices shape & dtype: {sphere.vertices.shape}, {sphere.vertices.dtype}")
print()

odfs shape & dtype: (110, 120, 724), float64
sphere.vertices shape & dtype: (724, 3), float64

